In [1]:
import os
from mistralai.client import MistralClient

In [2]:
api_key=os.environ.get('curelink')
client=MistralClient(api_key=api_key)

In [3]:
import requests,json
url='https://clchatagentassessment.s3.ap-south-1.amazonaws.com/queries.json'
response=requests.get(url)
chat_array=response.json()
# with open('chat', 'w') as f:

Make diet_chart more comprhensive,for better reponse time and cost 

In [4]:
fdays=[None,'Friday','Saturday','Sunday','Monday','Tuesday','Wednesday','Thrusday']
def extract_meal_plan(diet_chart):
    meal_plan = {}
    
    for day_data in diet_chart['meals_by_days']:
        day_order = day_data['order']
        day_meals = []
        
        for meal in day_data['meals']:
            meal_info = {
                'name': meal['name'],
                'order': meal['order'],
                'timings': meal['timings'],
                'food_items': []
            }
            
            for option in meal['meal_options']:
                food_items = [
                    item['Food']['name']
                    
                    for item in option['meal_option_food_items']
                ]
                
                meal_info['food_items'].append(food_items)
            
            day_meals.append(meal_info)
        
        meal_plan[f'{fdays[day_order]}'] = sorted(day_meals, key=lambda x: x['order'])
    
    return meal_plan

# Usage
final_data=[]
for i in range(len(chat_array)):
    diet_chart = chat_array[i]['profile_context']['diet_chart']  # Assuming chat_array is your list of patient data
    meal_plan = extract_meal_plan(diet_chart)
    meal_plan['program_name']=chat_array[0]['profile_context']['program_name']
    final_data.append(meal_plan)



Let's try ZERO SHOT

In [66]:
collective_response=[]
from datetime import datetime
model='mistral-large-latest'
def get_day_of_week(date_string):
    date_object = datetime.strptime(date_string, "%B %d, %Y, %I:%M %p")
    day_number = date_object.weekday()
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'] 
    return days[day_number]

In [67]:
for i in range(len(chat_array)):
    ele=chat_array[0]
    date_string = ele['chat_context']['ticket_created']
    day = get_day_of_week(date_string)
    relevant_food_plan=final_data[0][day]
    
    prompt = f"the chat history: {ele['chat_context']['chat_history']} \n.The diet plan {relevant_food_plan}. \n  "
    stream_response = client.chat(
        model = model,
        messages = [
            {
                
        "role": "system",
        "content": """You will be given recent chat between a patient and his dietician along with food chart for the day.If the person eats an hour before or after the scheduled time,its alright.
        If the person misses a food item to eat or if a person eats a different food or any extra food item (anytime in the day) ask him the reason .
        only if the person eats what is prescribed, appreciate him and tell him the benefits of the food item in just 1 sentence.If the person asks query in Romanized Hindi respond in Romanized Hindi as well.
        """
            },
            {
                "role":"user",
                "content": prompt
            
            },
            # {
            #     "role":"assistant",
            #     "content":"Great job on having your soaked methi seeds water! Methi seeds are excellent for managing blood sugar levels.I noticed you had a variety of dry fruits including almonds and walnuts. Could you tell me the reason for having these additional items?"
            # },
            # {
            #     "role":"user","content": " why didn't you ask him as to why did he consume fig and raisin if not presribed, mention the additional items "
            # }
        ]
        )

    collective_response.append({
        'ideal_reponse':f'{ele["ideal_response"]}', 'ticket_id':f'{ele["chat_context"]["ticket_id"]}','latest_query':f'{ele["latest_query"]}','generated_response': f'{stream_response.choices[0].message.content}'})


In [68]:
with open('my_json.json', 'w', encoding='utf-8') as f:
    json.dump(collective_response, f, ensure_ascii=False, indent=4)

I think lot of brainstroming and new and fresh perspective is needed